In [3]:
import linear_model as model
import src.project1.submit.AI as ai
from experiment.评估函数.simulator import Simulator
from experiment.评估函数.greedy_ai import GreedyAI
from experiment.评估函数.greedys import *
import numpy as np

chessboard_size = 8
time_out = 5


def greedy_fun(Vars):
    PVT, PVT_max = model.get_PVT_and_max(Vars)
    return lambda a, p: model.value_of_positions(a.reversi_env.updated_chessboard(a.chessboard, a.color, p), a.color,
                                                 PVT, PVT_max)


baselines = [random_baseline, middle_action_first, map_weight_sum]
baseline_cnt = len(baselines)
# 注意时间复杂度不要太高
落子次数 = lambda x : (baseline_cnt + x) * (baseline_cnt-1 + x) * 60
[落子次数(i) for i in range(2, 14)]  # 已知落子 60*30 = 1800 需要1.59276s
# 选择15s 一个温度，有 10000次以上，

[1200, 1800, 2520, 3360, 4320, 5400, 6600, 7920, 9360, 10920, 12600, 14400]

In [4]:
import random
import time

def simulated_annealing(state, T=1, log_interval=10, neighbour_cnt=10):
    start_time = time.time()
    t = 0  # time step
    f = []  # a recording of values
    while T >= 1e-7:
        vars_list = model.neighbours_continuous(state, -1, 1, 0.1, neighbour_cnt)
        funcs = baselines + list(map(greedy_fun, vars_list))
        length = len(funcs)
        matches = 2/2*length*(length-1)
        scores = np.zeros(length)
        for i in range(length):
            for j in range(i + 1, length):
                for i_color in [ai.COLOR_BLACK, ai.COLOR_WHITE]:
                    agents = {i_color: GreedyAI(chessboard_size, i_color, funcs[i]),
                              -i_color: GreedyAI(chessboard_size, -i_color, funcs[j])}
                    simulator = Simulator(chessboard_size, time_out, agents)
                    winner = simulator.quick_run(no_print=True)
                    if winner == i_color:
                        scores[i] += 1
                    elif winner == -i_color:
                        scores[j] += 1
        diff = np.inf
        new_idx = -1
        new_value = np.nan
        while not (diff > 0 or np.exp(diff / T) > np.random.uniform()):
            new_idx = random.randint(1, neighbour_cnt)
            new_value = scores[new_idx]
            diff = new_value - scores[0]

        state = vars_list[new_idx]
        f.append(new_value)
        # update time and temperature
        if t % log_interval == 0:
            print(f"step {t}: T={T}, current_value={scores[new_idx]/matches}, time used: {time.time()-start_time}")
            print(f"Current var is {state}")
        t += 1
        T = 0.999 ** t
    print(f"step {t}: T={T}, current_value={scores[new_idx]/matches}")
    return state, f

In [ ]:
import matplotlib.pyplot as plt

# define your own schedule and halt condition
solution, record = simulated_annealing(
    np.array([-1, 1, -0.3, -0.2, 0.7, -0.2, -0.2, -0.3, 0.4, -0.2])
    ,neighbour_cnt=4
)
solution

step 0: T=1, current_value=0.10714285714285714, time used: 4.05219030380249
Current var is [-0.89561378  1.         -0.19137715 -0.30143643  0.90671003 -0.24745839
 -0.14145017 -0.23293248  0.53537761 -0.12872306]
step 10: T=0.9900448802097482, current_value=0.10714285714285714, time used: 32.04831790924072
Current var is [-0.84644473  0.87280518  0.02534043 -0.3556755   0.97759364 -0.56621937
 -0.02188858 -0.27254562  0.98973532  0.16321912]
step 20: T=0.9801888648295347, current_value=0.03571428571428571, time used: 59.4508695602417
Current var is [-0.81753312  0.42876027 -0.29537373 -0.25230171  0.69330562 -0.49365052
  0.60343311 -0.32361499  0.64940336 -0.10335152]
step 30: T=0.9704309672630858, current_value=0.08928571428571429, time used: 87.02612280845642
Current var is [-0.75074206  0.30397095 -0.61978262 -0.80060076  0.37218739 -0.6072005
  0.99951262 -0.21673295  0.42623422 -0.22120175]
step 40: T=0.9607702107358118, current_value=0.05357142857142857, time used: 114.82277798

In [ ]:
# visualize the curve
plt.plot(record)
plt.xlabel("time step")
plt.ylabel("value")

## Deprecated: 

In [ ]:
def imerge(a, b):
    i1 = iter(a)
    i2 = iter(b)
    while True:
        try:
            yield i1.next()
            yield i2.next()
        except StopIteration:
            return


for i in imerge(range(10), range(8)):
    print(i)

In [ ]:
def schedule(t):
    T = 1000
    while True:
        yield T
        T *= 0.98


def simulated_annealing(InitialVars, rounds):
    current = InitialVars
    for t in range(rounds):
        T = schedule(t)
        if T <= 10:
            return current
        for var in imerge(current, model.neighbours(current))
